In [13]:
import requests
import numpy as np
import time

# Helper function to convert radius in meters to decimal degrees
def meters_to_decimal_degrees(meters, latitude):
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))

# Function to create a grid of locations around a central point
def create_location_grid(center_location, overall_radius, tile_radius):
    lat, lng = map(float, center_location.split(','))
    delta_deg = meters_to_decimal_degrees(overall_radius, lat)
    tile_delta_deg = meters_to_decimal_degrees(tile_radius * 2, lat)
    
    lat_steps = int(np.ceil(delta_deg / tile_delta_deg))
    lng_steps = int(np.ceil(delta_deg / tile_delta_deg))
    
    locations = []
    for i in range(-lat_steps, lat_steps + 1):
        for j in range(-lng_steps, lng_steps + 1):
            new_lat = lat + (i * tile_delta_deg)
            new_lng = lng + (j * tile_delta_deg)
            locations.append(f"{new_lat},{new_lng}")
    print(f"Generated {len(locations)} tiles for processing.")
    return locations

# Modified function to calculate costs with geographical tiling and unique place count
def calculate_cost_with_tiling(api_key, center_location, overall_radius, tile_radius, place_type):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    unique_place_ids = set()  # To track unique places
    locations = create_location_grid(center_location, overall_radius, tile_radius)
    
    for index, location in enumerate(locations):
        next_page_token = None
        print(f"Processing tile {index + 1}/{len(locations)} at location {location}")
        
        while True:
            params = {
                'location': location,
                'radius': tile_radius,
                'type': place_type,
                'key': api_key
            }
            
            if next_page_token:
                params['pagetoken'] = next_page_token
                time.sleep(2)
            
            response = requests.get(endpoint_url, params=params)
            data = response.json()
            
            if data['status'] == 'OK':
                for result in data['results']:
                    unique_place_ids.add(result['place_id'])
                next_page_token = data.get('next_page_token')
                if not next_page_token:
                    break
            else:
                print(f"Failed to retrieve data for tile {index + 1}: {data['status']}")
                break
        
        print(f"Tile {index + 1} completed. Unique places found so far: {len(unique_place_ids)}")
    
    total_places_count = len(unique_place_ids)
    cost_per_1000 = 17  # Assuming cost as per pricing information
    total_cost = (total_places_count / 1000) * cost_per_1000
    print(f"Estimated total cost for fetching details of {total_places_count} unique places: ${total_cost:.2f}")

api_key = 'yourapikey'  # Replace with your actual API key
center_location = '33.75556,-84.40000'
overall_radius = 15000  # Larger area to cover with tiling
tile_radius = 1000  # Radius for each tile, can be adjusted
place_type = 'restaurant'

calculate_cost_with_tiling(api_key, center_location, overall_radius, tile_radius, place_type)


Generated 289 tiles for processing.
Processing tile 1/289 at location 33.582686443793385,-84.57287355620663
Failed to retrieve data for tile 1: ZERO_RESULTS
Tile 1 completed. Unique places found so far: 0
Processing tile 2/289 at location 33.582686443793385,-84.5512643616808
Tile 2 completed. Unique places found so far: 4
Processing tile 3/289 at location 33.582686443793385,-84.52965516715497
Failed to retrieve data for tile 3: ZERO_RESULTS
Tile 3 completed. Unique places found so far: 4
Processing tile 4/289 at location 33.582686443793385,-84.50804597262915
Tile 4 completed. Unique places found so far: 10
Processing tile 5/289 at location 33.582686443793385,-84.48643677810331
Failed to retrieve data for tile 5: ZERO_RESULTS
Tile 5 completed. Unique places found so far: 10
Processing tile 6/289 at location 33.582686443793385,-84.46482758357749
Tile 6 completed. Unique places found so far: 31
Processing tile 7/289 at location 33.582686443793385,-84.44321838905167
Tile 7 completed. Uniqu

In [15]:
import requests
import pandas as pd
import numpy as np
import time

# Your API Key and initial parameters
api_key = 'youdapikey'  # Replace with your actual API key
center_location = '33.75556,-84.40000'  # Center point for the search
overall_radius = 15000  # Larger area to cover
tile_radius = 1000  # Smaller tiles for each query
type = 'restaurant'

def meters_to_decimal_degrees(meters, latitude):
    return meters / (111.32 * 1000 * np.cos(latitude * (np.pi / 180)))

def create_location_grid(center_location, overall_radius, tile_radius):
    lat, lng = [float(coord) for coord in center_location.split(',')]
    delta_deg = meters_to_decimal_degrees(overall_radius, lat)
    tile_delta_deg = meters_to_decimal_degrees(tile_radius * 2, lat)
    
    lat_steps = int(np.ceil(delta_deg / tile_delta_deg))
    lng_steps = int(np.ceil(delta_deg / tile_delta_deg))
    
    locations = []
    for i in range(-lat_steps, lat_steps + 1):
        for j in range(-lng_steps, lng_steps + 1):
            new_lat = lat + (i * tile_delta_deg)
            new_lng = lng + (j * tile_delta_deg)
            locations.append(f"{new_lat},{new_lng}")
    return locations

def fetch_places_for_tile(api_key, location, radius, type, unique_places_set):
    places = []
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        'location': location,
        'radius': radius,
        'type': type,
        'key': api_key
    }
    print(f"Fetching places for tile centered at {location}")
    response = requests.get(endpoint_url, params=params)
    data = response.json()
    
    if data['status'] == 'OK':
        for place in data['results']:
            place_id = place.get('place_id')
            if place_id not in unique_places_set:
                unique_places_set.add(place_id)
                place_info = {
                    'name': place.get('name'),
                    'latitude': place.get('geometry', {}).get('location', {}).get('lat'),
                    'longitude': place.get('geometry', {}).get('location', {}).get('lng'),
                    'place_id': place_id,
                }
                places.append(place_info)
                print(f"Added place: {place_info['name']}")
    else:
        print(f"Failed to fetch places for tile at {location}: {data['status']}")
    return places

def fetch_all_places(api_key, center_location, overall_radius, tile_radius, type):
    all_places = []
    unique_places_set = set()
    locations = create_location_grid(center_location, overall_radius, tile_radius)
    print(f"Total tiles to process: {len(locations)}")
    
    for idx, location in enumerate(locations):
        print(f"Processing tile {idx + 1}/{len(locations)}")
        tile_places = fetch_places_for_tile(api_key, location, tile_radius, type, unique_places_set)
        all_places.extend(tile_places)
        time.sleep(0.1)  # Small delay to manage API call rate
        
    return all_places

# Fetch all places with geographical tiling
all_places = fetch_all_places(api_key, center_location, overall_radius, tile_radius, type)

# Convert the list of places into a DataFrame
df = pd.DataFrame(all_places)

# Specify the file name (adjust the path as needed)
file_name = 'C:/Users/agutierrez/Downloads/places_data.xlsx'  # Adjusted for execution environment

# Export the DataFrame to an Excel file
df.to_excel(file_name, index=False)

print(f"Data saved to {file_name}")


Total tiles to process: 289
Processing tile 1/289
Fetching places for tile centered at 33.582686443793385,-84.57287355620663
Failed to fetch places for tile at 33.582686443793385,-84.57287355620663: ZERO_RESULTS
Processing tile 2/289
Fetching places for tile centered at 33.582686443793385,-84.5512643616808
Added place: The Historic Green Manor Restaurant
Added place: 1804 Pizza Bar & Lounge
Added place: The PitStop
Added place: PescaVegan Life 2
Processing tile 3/289
Fetching places for tile centered at 33.582686443793385,-84.52965516715497
Failed to fetch places for tile at 33.582686443793385,-84.52965516715497: ZERO_RESULTS
Processing tile 4/289
Fetching places for tile centered at 33.582686443793385,-84.50804597262915
Added place: Waffle House
Added place: New Jack City
Added place: Circle K
Added place: A Taste of the Island
Added place: King's Fish & Wings
Added place: D'lish Restaurant and Catering
Processing tile 5/289
Fetching places for tile centered at 33.582686443793385,-84.